In [105]:
def extract_all_company_data(file_path):
    csv_files = []  # List to store .csv file names

    # Get a list of all files and directories in the specified directory
    for file in os.listdir(file_path):
        if file.endswith(".csv"):
            csv_path = os.path.join(file_path, file)
            df = pd.read_csv(csv_path)
            print(f'{file}: {len(df)} rows')

    company_data = []

    # Iterate through the CSV files
    for csv_file in csv_files:
        # Read the CSV file into a DataFrame
        csv_path = os.path.join(file_path, csv_file)
        df = pd.read_csv(csv_path)

        # Append each row to the company_data list
        for index, row in df.iterrows():
            company_data.append(row.to_dict())  # Convert each row to a dictionary and append

    return pd.DataFrame(company_data)

# Call the modified function to print row counts for each CSV
extract_all_company_data(file_path)

audi.csv: 10668 rows
bmw.csv: 10781 rows
cclass.csv: 3899 rows
focus.csv: 5454 rows
ford.csv: 17965 rows
hyundi.csv: 4860 rows
merc.csv: 13119 rows
skoda.csv: 6267 rows
toyota.csv: 6738 rows
unclean cclass.csv: 4006 rows
unclean focus.csv: 5604 rows
vauxhall.csv: 13632 rows
vw.csv: 15157 rows


""


In [107]:
import os
import pandas as pd

def extract_all_company_data(file_path_read, file_path_write, exclude_files):
    csv_files = []  # List to store .csv file names
    
    # Get a list of all files and directories in the specified directory
    for file in os.listdir(file_path_read):
        if file.endswith(".csv") and file not in exclude_files:
            csv_files.append(file)  # Add .csv file to the list
    
    company_data = []
    
    # Iterate through the CSV files
    for csv_file in (csv_files[:2]):
        # Read the CSV file into a DataFrame
        csv_path = os.path.join(file_path_read, csv_file)
        df = pd.read_csv(csv_path)
        
        # Get the manufacturer name (assuming the CSV filename is the manufacturer name)
        manufacturer_name = os.path.splitext(os.path.basename(csv_file))[0]
        
        # Add a column for the manufacturer name
        df['manufacturer'] = manufacturer_name
        
        # Save the modified DataFrame to a new CSV file in the 'selected' folder
        output_file_path = os.path.join(file_path_write, f"{manufacturer_name}_modified.csv")
        df.to_csv(output_file_path, index=False)
        
        # Append each row to the company_data list
        for index, row in df.iterrows():
            company_data.append(row.to_dict())  # Convert each row to a dictionary and append
        
    return pd.DataFrame(company_data)

# Define the path to read from
file_path_read = 'data/raw'
# Define the path to write to (inside the 'selected' folder)
file_path_write = 'data/selected'
# Define the files to be excluded
exclude_files = ['cclass.csv', 'focus.csv', 'unclean cclass.csv', 'unclean focus.csv']

# Call the function to process and save each individual CSV
extract_all_company_data(file_path_read, file_path_write, exclude_files)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacturer
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...
21444,X3,2016,19000,Automatic,40818,Diesel,150,54.3,2.0,bmw
21445,5 Series,2016,14600,Automatic,42947,Diesel,125,60.1,2.0,bmw
21446,3 Series,2017,13100,Manual,25468,Petrol,200,42.8,2.0,bmw
21447,1 Series,2014,9930,Automatic,45000,Diesel,30,64.2,2.0,bmw


In [110]:
import pandas as pd

# Load the 'audi.csv' file into a DataFrame
audi_df = pd.read_csv('data/selected/bmw_modified.csv')  # Replace with the actual path to your 'audi.csv' file

# Check the number of rows in the 'audi.csv' DataFrame
num_rows = audi_df.shape[0]  # The number of rows is the first element of the shape tuple

print('Number of rows in audi.csv:', num_rows)


Number of rows in audi.csv: 10781
